## Two-area network with a stochastic load

The purpose of this notebook is to show how pan can be used to simulate the two-area power network with a stochastic load. The behavior of the load is described by two Julia functions and the time series defining the stochastic load is also created in Julia.

A single line schematic of Kundur's two area network is shown below:

<a href="https://www.researchgate.net/figure/Kundurs-two-area-power-system_fig1_281703659"><img src="https://www.researchgate.net/profile/Zahra-Rafi/publication/281703659/figure/fig1/AS:338164796411905@1457636178242/Kundurs-two-area-power-system.png" alt="Kundur's two-area power system"/></a>

In the netlist used here, the loads connected at buses 7 and 9 are stochastic, i.e., their values are described by an Ornstein-Uhlenbeck process.

In [ ]:
using Format: format, printfmtln
using Printf: @printf
using Random
using Statistics
using DSP
using PAN
using Printf
using HDF5
import LinearAlgebra

The [netlist file](two-area.pan) contains the following line:

`model STLOAD nport macro=yes setup="stoch_load_setup" evaluate="stoch_load_eval"`

which instructs pan to look for two Julia functions called `stoch_load_setup` and `stoch_load_eval` when initializing and evaluating the N-port instance, respectively. The initialization function must return a dictionary with the parameters names as keys and their default value as values, while the evaluation function must return the implicit equations describing the N-port and their derivative w.r.t. currents and voltages. 
    
For a more detailed description of how to implement an N-port device in Julia, see [this notebook](../NPort.ipynb).

In [ ]:
function stoch_load_setup()
    return (P = 1e6, )
end;

function stoch_load_eval(n::Number, V::AbstractArray, I::AbstractArray, time::Number, parameters...)
    MIN_MAGNITUDE = 200e3 ^ 2
    P = parameters[1] * V[3] # active power
    Q = 0
    magnitude = V[1]^2 + V[2]^2
    if magnitude < MIN_MAGNITUDE
        magnitude = MIN_MAGNITUDE
    end
    magnitude_squared = magnitude ^ 2
    f = [
        I[1] - (V[1] * P + V[2] * Q) / magnitude,
        I[2] - (V[2] * P - V[1] * Q) / magnitude,
        I[3]
    ]

    C = zeros(n, n)
    C[1,1] = -( P / magnitude - (V[1] * P + V[2] * Q) * 2 * V[1] / magnitude_squared)
    C[1,2] = -( Q / magnitude - (V[1] * P + V[2] * Q) * 2 * V[2] / magnitude_squared)
    C[1,3] = -(parameters[1] * V[1] / magnitude)
    C[2,1] = -(-Q / magnitude - (V[2] * P - V[1] * Q) * 2 * V[1] / magnitude_squared)
    C[2,2] = -( P / magnitude - (V[2] * P - V[1] * Q) * 2 * V[2] / magnitude_squared)
    C[2,3] = -(parameters[1] * V[2] / magnitude)

    R = Matrix{Float64}(LinearAlgebra.I, n, n)

    return f, C, R
end;

Here we define the two functions that are used to generate the Ornstein-Uhlenbeck process and to build a two-row matrix containing the time instants and the OU samples.

In [ ]:
function OU(dt::Number, alpha::Number, mu::Number, sigma::Number, N::Integer; rng = nothing)
    coeff = [alpha * mu * dt, 1 / (1 + alpha * dt)]
    if rng != nothing
        rnd = sigma * sqrt(dt) * randn(rng, N)
    else
        rnd = sigma * sqrt(dt) * randn(N)
    end
    ou = zeros(N)
    ou[1] = mu
    for i = 1 :  N-1
        ou[i+1] = (ou[i] + coeff[1] + rnd[i]) * coeff[2]
    end
    return ou
end;

function make_noise_samples(tstart::Number, tend::Number, dt::Number, alpha::Number,
                            mu::Number, sigma::Number, seed::Number)
    t = dt .+ (tstart : dt : tstop)
    N_samples = length(t)
    rng = MersenneTwister(seed)
    noise_samples = zeros(2, N_samples)
    noise_samples[1,:] = t
    noise_samples[2,:] = OU(dt, alpha, mu, sigma, N_samples, rng = rng)
    return noise_samples
end;

The following two lines in the [netlist file](two-area.pan)

`V7   rnd7   gnd  vsource wave="noise_samples_bus_7"` <br/>
`V9   rnd9   gnd  vsource wave="noise_samples_bus_9"`

tell pan that it should look for two variables (called `noise_samples_bus_7` and `noise_samples_bus_9`) in Julia memory to use as waveforms for the `V7` and `V9` voltage source objects.

We therefore build these two variables by calling the `make_noise_sample` function defined above.

In [ ]:
alpha = 0.5
mu = 0
sigma = 0.5
frand = 10
dt = 1 / frand
tstart = 0
tstop = 1 * 60 * 60

noise_samples_bus_7 = make_noise_samples(tstart, 1.01 * tstop, dt, alpha, mu, sigma, 1234567)
noise_samples_bus_9 = make_noise_samples(tstart, 1.01 * tstop, dt, alpha, mu, sigma, 6234160);

### Load the netlist

In [ ]:
netlist_file = "two-area.pan"
ok, libs = load_netlist(netlist_file)
if ! ok
    println("load_netlist failed.")
end

### Run a transient analysis

By defining `mem_vars`, we ask pan to return the values of these variables saved to memory when `tran` returns. In this way, we don't have to load the same data from a file.

In [ ]:
mem_vars = ["time", "omega1", "omega2", "omega3", "omega4"]
data = tran("Tr", tstop, mem_vars, libs, nettype=1, method=1, timepoints=1/frand, forcetps=1)

### Compute the PSD of the rotors' angular velocities

In [ ]:
time = data[:, 1]
omega = 60 * data[:, 2:end]
n_omega = size(omega)[2]
ps = [welch_pgram(omega[:,i], fs=frand) for i = 1:n_omega];

### Plot the results

In [ ]:
using PyPlot
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.family"] = "serif"
rcParams["font.serif"] = ["Times New Roman"]
figure(figsize=(8.5 / 2.54, 5 / 2.54))
fig = gcf()
ax = gca()
ax.set_position([0.2, 0.2, 0.75, 0.75])
cmap = ("black", "red", "green", "blue")
for i = 1:n_omega
    plot(ps[i].freq, pow2db.(ps[i].power), color=cmap[i], label=latexstring("\\omega_" * string(i)), linewidth=1)
end
ax.grid(which="major", axis="y", linewidth=0.5, linestyle=":")
for side in ("right","top")
    ax.spines[side].set_visible(false)
end
ax.set_xlabel("Frequency [Hz]", fontsize=8)
ax.set_ylabel("Power [dB]", fontsize=8)
ax.legend(loc="lower left", fontsize=8)
ax.set_ylim([-125, -60])
ax.tick_params(axis="x", labelsize=8)
ax.tick_params(axis="y", labelsize=8)
savefig("stochastic_load.pdf")

### Save the results to disk

In [ ]:
freq = collect([[f for f in ps[i].freq] for i = 1 : length(ps)])
Pxx = collect([[p for p in pow2db.(ps[i].power)] for i = 1 : length(ps)])
for i = 1 : length(freq)
    h5write("stochastic_load.h5", "data/freq_" * string(i), freq[i])
    h5write("stochastic_load.h5", "data/Pxx_" * string(i), Pxx[i])
end